# Homework1: 私有数据微调Chatglm3
扩大训练数据集语料，并使用LangChain+GPT-3.5 Turbo合成新增的训练数据，使用QLoRA在ChatGLM3-6B上微调，并观察Loss变化情况，对比微调前后模型生成内容质量

## 打印 PyTorch 的配置信息和第一个 CUDA 设备的属性

In [1]:
import torch

print(torch.__config__.show(), torch.cuda.get_device_properties(0))

PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.3.6 (Git Hash 86e6af5974177e513fd3fee58425e1063e7f1361)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 12.1
  - NVCC architecture flags: -gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_75;-gencode;arch=compute_80,code=sm_80;-gencode;arch=compute_86,code=sm_86;-gencode;arch=compute_90,code=sm_90
  - CuDNN 8.9.2
  - Magma 2.6.1
  - Build settings: BLAS_INFO=mkl, BUILD_TYPE=Release, CUDA_VERSION=12.1, CUDNN_VERSION=8.9.2, CXX_COMPILER=/opt/rh/devtoolset-9/root/usr/bin/c++, CXX_FLAGS= -D_GLIBCXX_USE_CXX11_ABI=0 -fabi-version=11 -fvisibility-inlines-hidden -DUSE_PTHREADPOOL -DNDEBUG -DUSE_KINETO 

## 定义全局变量和参数

In [2]:
# 定义全局变量和参数
model_name_or_path = 'THUDM/chatglm3-6b'  # 模型ID或本地路径
# train_data_path = 'data/zhouyi_dataset_handmade.csv'    # 训练数据路径
train_data_path = 'data/zhouyi_dataset_20240803_172121.csv'    # 训练数据路径(批量生成数据集）
eval_data_path = None                     # 验证数据路径，如果没有则设置为None
seed = 8                                 # 随机种子
max_input_length = 512                    # 输入的最大长度
max_output_length = 1536                  # 输出的最大长度
lora_rank = 16                             # LoRA秩
lora_alpha = 32                           # LoRA alpha值
lora_dropout = 0.05                       # LoRA Dropout率
prompt_text = ''                          # 所有数据前的指令文本

## 数据处理

In [3]:
from datasets import load_dataset

dataset = load_dataset("csv", data_files=train_data_path)
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['content', 'summary'],
        num_rows: 400
    })
})


In [4]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [5]:
show_random_elements(dataset["train"], num_examples=5)

,content,summary
0,坤卦与其他文化中的象征有何异同？,坤卦是周易卦象中的一种，代表大吉大利。它由两个坤卦叠加而成，象征着地势平坦、顺承天道。坤卦代表着顺从和承载万物，预示着在正确的方向上前行，依循正道将带来吉利。此卦暗示着前往西南会有益，而东北则要防范丧失。总体而言，坤卦指示着稳定和吉祥的运势。\n\n周易学者解释，坤卦的核心哲学是柔顺和厚载之德，它提醒人们要顺应运势，保持安宁，避免冒险。同时，坤卦也预示着需要依靠合作，共同完成事业。在家庭、事业、交际等方面，都需要稳健地前行，降低冒险。此外，坤卦还暗示着适时的静制动，以达到事业的成功。\n\n在传统解卦中，坤卦代表地道生育和顺从天道的精神，预示着诸事顺利，但需要避免冒险急进。应注重内心修养，积蓄养德，以温和、谨慎的态度对待事业、经商、婚姻等方面。切忌贪功自傲，而是要持之以恒，谨慎行事。
1,乾卦的象征意义有哪些？,在周易中，乾卦代表着天，是六十四卦之首。这一卦象由六个阳爻组成，意味着强大、兴盛和健康。乾卦预示着大吉大利，并教导人们遵守天道的原则。\n\n乾卦的核心哲学是：天道刚健，运行不已。君子观此卦象，应效法天之刚健，自强不息。\n\n根据北宋易学家邵雍的解释，得此卦者将迎来名利双收之象，应把握机会，争取成果。然而，也要警惕过于强硬的行为，以免衰败。\n\n在传统解卦中，乾卦预示着事业、经商、求名等方面的大吉大利，但需要冷静处世、保持谦逊和平和的心境，方能充分发挥才智，取得成功。对于婚恋关系，乾卦提示阳盛阴衰，建议柔情可以相济，形成美满结果。在决策方面，乾卦的核心要义是坚持刚健、正直、公允的实质，修养德行，积累知识，坚定信念，自强不息，从而能克服困难，消除灾难。
2,坤卦的历史典故有哪些？,坤卦是周易卦象中的一种，代表大吉大利。它由两个坤卦叠加而成，象征着地势平坦、顺承天道。坤卦代表着顺从和承载万物，预示着在正确的方向上前行，依循正道将带来吉利。此卦暗示着前往西南会有益，而东北则要防范丧失。总体而言，坤卦指示着稳定和吉祥的运势。\n\n周易学者解释，坤卦的核心哲学是柔顺和厚载之德，它提醒人们要顺应运势，保持安宁，避免冒险。同时，坤卦也预示着需要依靠合作，共同完成事业。在家庭、事业、交际等方面，都需要稳健地前行，降低冒险。此外，坤卦还暗示着适时的静制动，以达到事业的成功。\n\n在传统解卦中，坤卦代表地道生育和顺从天道的精神，预示着诸事顺利，但需要避免冒险急进。应注重内心修养，积蓄养德，以温和、谨慎的态度对待事业、经商、婚姻等方面。切忌贪功自傲，而是要持之以恒，谨慎行事。
3,比卦对现代生活有何启示？,在周易中，比卦是一个极具深意的卦象，由两个异卦相叠组成：下卦坤（地）和上卦坎（水）。这一卦象表示相亲相依之意，暗示长期相互依赖将会无灾无咎，因此预示着吉利之事。同时也暗示了不愿臣服的邦国来朝，迟迟不来者将会有难。\n\n在这个卦象中，坎为水，坤为地，地纳水的形象象征着相亲相依、亲密无间，因此先王从中取法于水附大地，地纳江河之象，以此来建立万国，亲近诸侯。\n\n比卦的哲学内涵是宽宏无私、精诚团结，预示着平顺的运势，可以借助贵人的力量，得到成功。在事业上，需要以诚实、信任的态度去待人，以宽厚、正直的心胸来面对自己的工作和人际关系。在经商上，需与他人密切合作，遵守商业道德，如唯利是图、贪心不足会导致损失。在婚恋中，暗示着美好的姻缘，忠诚和相互尊重将会带来长久的幸福。\n\n总的来说，比卦传达着待人善良、多帮助他人、借力求存的哲学思想，让人在面对人际关系和事业发展中更加谨慎和坚定。
4,比卦与传统文化有何联系？,在周易中，比卦是一个极具深意的卦象，由两个异卦相叠组成：下卦坤（地）和上卦坎（水）。这一卦象表示相亲相依之意，暗示长期相互依赖将会无灾无咎，因此预示着吉利之事。同时也暗示了不愿臣服的邦国来朝，迟迟不来者将会有难。\n\n在这个卦象中，坎为水，坤为地，地纳水的形象象征着相亲相依、亲密无间，因此先王从中取法于水附大地，地纳江河之象，以此来建立万国，亲近诸侯。\n\n比卦的哲学内涵是宽宏无私、精诚团结，预示着平顺的运势，可以借助贵人的力量，得到成功。在事业上，需要以诚实、信任的态度去待人，以宽厚、正直的心胸来面对自己的工作和人际关系。在经商上，需与他人密切合作，遵守商业道德，如唯利是图、贪心不足会导致损失。在婚恋中，暗示着美好的姻缘，忠诚和相互尊重将会带来长久的幸福。\n\n总的来说，比卦传达着待人善良、多帮助他人、借力求存的哲学思想，让人在面对人际关系和事业发展中更加谨慎和坚定。


In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,
                                          trust_remote_code=True,
                                          revision='b098244')

/root/miniconda3/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [7]:
# tokenize_func 函数
def tokenize_func(example, tokenizer, ignore_label_id=-100):
    """
    对单个数据样本进行tokenize处理。

    参数:
    example (dict): 包含'content'和'summary'键的字典，代表训练数据的一个样本。
    tokenizer (transformers.PreTrainedTokenizer): 用于tokenize文本的tokenizer。
    ignore_label_id (int, optional): 在label中用于填充的忽略ID，默认为-100。

    返回:
    dict: 包含'tokenized_input_ids'和'labels'的字典，用于模型训练。
    """

    # 构建问题文本
    question = prompt_text + example['content']
    if example.get('input', None) and example['input'].strip():
        question += f'\n{example["input"]}'

    # 构建答案文本
    answer = example['summary']

    # 对问题和答案文本进行tokenize处理
    q_ids = tokenizer.encode(text=question, add_special_tokens=False)
    a_ids = tokenizer.encode(text=answer, add_special_tokens=False)

    # 如果tokenize后的长度超过最大长度限制，则进行截断
    if len(q_ids) > max_input_length - 2:  # 保留空间给gmask和bos标记
        q_ids = q_ids[:max_input_length - 2]
    if len(a_ids) > max_output_length - 1:  # 保留空间给eos标记
        a_ids = a_ids[:max_output_length - 1]

    # 构建模型的输入格式
    input_ids = tokenizer.build_inputs_with_special_tokens(q_ids, a_ids)
    question_length = len(q_ids) + 2  # 加上gmask和bos标记

    # 构建标签，对于问题部分的输入使用ignore_label_id进行填充
    labels = [ignore_label_id] * question_length + input_ids[question_length:]

    return {'input_ids': input_ids, 'labels': labels}


In [8]:
column_names = dataset['train'].column_names
tokenized_dataset = dataset['train'].map(
    lambda example: tokenize_func(example, tokenizer),
    batched=False, 
    remove_columns=column_names
)

In [9]:
tokenized_dataset = tokenized_dataset.shuffle(seed=seed)
tokenized_dataset = tokenized_dataset.flatten_indices()

In [10]:
import torch
from typing import List, Dict, Optional

# DataCollatorForChatGLM 类
class DataCollatorForChatGLM:
    """
    用于处理批量数据的DataCollator，尤其是在使用 ChatGLM 模型时。

    该类负责将多个数据样本（tokenized input）合并为一个批量，并在必要时进行填充(padding)。

    属性:
    pad_token_id (int): 用于填充(padding)的token ID。
    max_length (int): 单个批量数据的最大长度限制。
    ignore_label_id (int): 在标签中用于填充的ID。
    """

    def __init__(self, pad_token_id: int, max_length: int = 2048, ignore_label_id: int = -100):
        """
        初始化DataCollator。

        参数:
        pad_token_id (int): 用于填充(padding)的token ID。
        max_length (int): 单个批量数据的最大长度限制。
        ignore_label_id (int): 在标签中用于填充的ID，默认为-100。
        """
        self.pad_token_id = pad_token_id
        self.ignore_label_id = ignore_label_id
        self.max_length = max_length

    def __call__(self, batch_data: List[Dict[str, List]]) -> Dict[str, torch.Tensor]:
        """
        处理批量数据。

        参数:
        batch_data (List[Dict[str, List]]): 包含多个样本的字典列表。

        返回:
        Dict[str, torch.Tensor]: 包含处理后的批量数据的字典。
        """
        # 计算批量中每个样本的长度
        len_list = [len(d['input_ids']) for d in batch_data]
        batch_max_len = max(len_list)  # 找到最长的样本长度

        input_ids, labels = [], []
        for len_of_d, d in sorted(zip(len_list, batch_data), key=lambda x: -x[0]):
            pad_len = batch_max_len - len_of_d  # 计算需要填充的长度
            # 添加填充，并确保数据长度不超过最大长度限制
            ids = d['input_ids'] + [self.pad_token_id] * pad_len
            label = d['labels'] + [self.ignore_label_id] * pad_len
            if batch_max_len > self.max_length:
                ids = ids[:self.max_length]
                label = label[:self.max_length]
            input_ids.append(torch.LongTensor(ids))
            labels.append(torch.LongTensor(label))

        # 将处理后的数据堆叠成一个tensor
        input_ids = torch.stack(input_ids)
        labels = torch.stack(labels)

        return {'input_ids': input_ids, 'labels': labels}

In [11]:
# 准备数据整理器
data_collator = DataCollatorForChatGLM(pad_token_id=tokenizer.pad_token_id)

## 加载模型

In [12]:
from transformers import AutoModel, BitsAndBytesConfig

_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])
# 加载量化后模型
model = AutoModel.from_pretrained(model_name_or_path,
                                  quantization_config=q_config,
                                  device_map='auto',
                                  trust_remote_code=True,
                                  revision='b098244')

model.supports_gradient_checkpointing = True  
model.gradient_checkpointing_enable()
model.enable_input_require_grads()

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

/root/miniconda3/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.


In [13]:
from peft import TaskType, LoraConfig, get_peft_model, prepare_model_for_kbit_training
from peft.utils import TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING

kbit_model = prepare_model_for_kbit_training(model)
target_modules = TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING['chatglm']

You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.


In [14]:
lora_config = LoraConfig(
    target_modules=target_modules,
    r=lora_rank,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    bias='none',
    inference_mode=False,
    task_type=TaskType.CAUSAL_LM
)

In [15]:
qlora_model = get_peft_model(kbit_model, lora_config)
qlora_model.print_trainable_parameters()

trainable params: 3,899,392 || all params: 6,247,483,392 || trainable%: 0.06241540401681151


## QLoRA 微调模型

In [16]:
import datetime

timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

train_epochs = 3
output_dir = f"models/{model_name_or_path}-epoch{train_epochs}-{timestamp}"

In [17]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir=output_dir,                            # 输出目录
    per_device_train_batch_size=10,                     # 每个设备的训练批量大小
    gradient_accumulation_steps=1,                     # 梯度累积步数
    learning_rate=1e-3,                                # 学习率
    num_train_epochs=train_epochs,                     # 训练轮数
    lr_scheduler_type="linear",                        # 学习率调度器类型
    warmup_ratio=0.1,                                  # 预热比例
    logging_steps=5,                                 # 日志记录步数
    save_strategy="steps",                             # 模型保存策略
    save_steps=10,                                     # 模型保存步数
    # save_total_limit=5,                                # checkpoint上限
    optim="adamw_torch",                               # 优化器类型
    fp16=True,                                        # 是否使用混合精度训练
)


In [18]:
trainer = Trainer(
        model=qlora_model,
        args=training_args,
        train_dataset=tokenized_dataset,
        data_collator=data_collator
    )

In [19]:
trainer.train()

/root/miniconda3/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
5,4.479500
10,3.714500
15,2.755100
20,1.448800
25,0.504100
30,0.094000
35,0.024200
40,0.010900
45,0.006200
50,0.003600


/root/miniconda3/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/root/miniconda3/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/root/miniconda3/lib/python3.10/site-packages/torch/utils/checkpoint.p

TrainOutput(global_step=120, training_loss=0.5441284704681796, metrics={'train_runtime': 164.0753, 'train_samples_per_second': 7.314, 'train_steps_per_second': 0.731, 'total_flos': 1.299678903042048e+16, 'train_loss': 0.5441284704681796, 'epoch': 3.0})

In [20]:
trainer.model.save_pretrained(output_dir)

## 模型推理 - 使用 QLoRA 微调后的 ChatGLM-6B

In [1]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
model_name_or_path = 'THUDM/chatglm3-6b'

In [2]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])

# 加载量化后模型(与微调的 revision 保持一致）
base_model = AutoModel.from_pretrained(model_name_or_path,
                                      quantization_config=q_config,
                                      device_map='auto',
                                      trust_remote_code=True,
                                      revision='b098244')

/root/miniconda3/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/root/miniconda3/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [3]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,
                                          trust_remote_code=True,
                                          revision='b098244')

In [5]:
from peft import PeftModel, PeftConfig

epochs = 3
timestamp = "20240804_112220"

peft_model_path = f"models/{model_name_or_path}-epoch{epochs}-{timestamp}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag=f"ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-{timestamp}"

### 微调前后效果对比

In [6]:
def compare_chatglm_results(query, base_model, qlora_model, training_tag):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = qlora_model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\n微调后（{training_tag}）：\n{ft_response}")
    return base_response, ft_response

In [7]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model, training_tag)

问题：解释下乾卦是什么？

原始输出：
乾卦是周易卦象中的一种，代表大吉大利。它由六个阳爻组成，意味着强大、兴盛。这一卦象预示着大吉大利，适合从事事业和交往。在面对此卦时，应充满信心，勇敢前进，以确保成功。

乾卦的核心哲学是：天行健，君子以自强不息。君子应像乾卦那样，坚定自信，勇敢前进， volvement和亨通。

在传统解卦中，乾卦代表阳刚之德，预示着事业吉利，奴隶主主人的事业将会顺利发展，主人心悦诚服，将会得到成功。然而，也要避免过于强硬，而是要刚中带柔，才能真正发挥才干。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20240804_112220）：
[gMASK]sop 解释下乾卦是什么？ 在周易中，乾卦代表着天，是六十四卦之首。这一卦象由六个阳爻组成，意味着强大、兴盛和健康。乾卦预示着大吉大利，并教导人们遵守天道的原则。

乾卦的核心哲学是：天道刚健，运行不已。君子观此卦象，应效法天之刚健，自强不息。

根据北宋易学家邵雍的解释，得此卦者将迎来名利双收之象，应把握机会，争取成果。然而，也要警惕过于强硬的行为，以免衰败。

在传统解卦中，乾卦预示着事业、经商、求名等方面的大吉大利，但需要冷静处世、保持谦逊和平和的心境，方能充分发挥才智，取得成功。对于婚恋关系，乾卦提示阳盛阴衰，建议柔情可以相济，形成美满结果。在决策方面，乾卦的核心要义是坚持刚健、正直、公允的实质，修养德行，积累知识，坚定信念，自强不息，从而能克服困难，消除灾难。


In [8]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model, training_tag)

问题：周易中的讼卦是什么

原始输出：
{'name': '讼卦是周易卦象中的一种，代表正义与邪恶之间的斗争。它由两个相反的卦象乾（天）和坎（水）组成。乾为刚，坎为险，相辅相成，必然会产生争端。这一卦象表示虽有利可图，但需要警惕戒惧。起初会有吉利，但最终会有凶险。在面对此卦时，宜见贵族王公，而不宜涉水渡河。', 'content': '\n讼卦的象辞描述天与水相反，两者相背而行，预示着事理乖舛，即将出现争讼。君子观此卦象，应当杜绝争讼之事，谨慎行事。传统解卦认为，讼卦象征事业起初顺利，后来受挫，务必警惕戒惧，不得固执已见。经商方面则建议遵循公正、公平、互利的原则，避免发生冲突。在婚恋方面，双方应以温和的方式处理生活。对于决策而言，宜承认现实，顺其自然，知足，适可而止。'}


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20240804_112220）：
[gMASK]sop 周易中的讼卦是什么？ 在周易中，讼卦是一个充满深意的卦象。它由两个相反的卦象乾（天）和坎（水）组成。乾为刚，坎为险，相辅相成，必然会产生争端。这一卦象表示虽有利可图，但需要警惕戒惧。起初会有吉利，但最终会有凶险。在面对此卦时，宜见贵族王公，而不宜涉水渡河。

讼卦的象辞描述天与水相反，两者相背而行，预示着事理乖舛，即将出现争讼。君子观此卦象，应当杜绝争讼之事，谨慎行事。传统解卦认为，讼卦象征事业起初顺利，后来受挫，务必警惕戒惧，不得固执已见。经商方面则建议遵循公正、公平、互利的原则，避免发生冲突。在婚恋方面，双方应以温和的方式处理生活。对于决策而言，宜承认现实，顺其自然，知足，适可而止。


In [10]:
base_response, ft_response = compare_chatglm_results("师卦是什么？", base_model, qlora_model, training_tag)

问题：师卦是什么？

原始输出：
在周易中，师卦是一个极具深意的卦象，它由两个异卦相叠组成：下卦坎（水）和上卦坤（地）。这一卦象代表“师”，即军队，寓意着兵力和农力的结合。在这里，坎卦象征着水和险难，而坤卦象征着地和顺从，暗示着通过将军事力量安置于民间，可以在必要时顺利调动。

师卦的核心哲学是：虽然兵力代表着危险和战争，但其使用应当是圣人不得已而为之的最后手段。在正确的情况下，军事力量可以顺应形势，将危险转化为吉祥。因此，在军事策略上，此卦象征着出征将会顺利，无灾祸。

师卦紧随讼卦（争讼卦），在《序卦》中解释为“讼必有众起，故受之以师”。这意味着争端激化至众多人群的参与，形成了类似军队的集体力量。

根据《象辞》，此卦地中有水，呈现出军队聚众的形象。君子观此卦象，取法于容纳江河的大地，收容和畜养大众。这暗示着在困难重重的大环境下，需要包容别人，充满共情和善意。

师卦的各种解释中，都强调了困难重重、变化无穷的特点，以及对于团结力量和领导者正道的重视。在求名、婚恋和决策方面，也都强调了务实、慎重和坚持的态度，以及对正道、行为的坚守。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20240804_112220）：
[gMASK]sop 师卦是什么？ 在周易中，师卦是一个极具深意的卦象，它由两个异卦相叠组成：下卦坎（水）和上卦坤（地）。这一卦象代表“师”，即军队，寓意着兵力和农力的结合。在这里，坎卦象征着水和险难，而坤卦象征着地和顺从，暗示着通过将军事力量安置于民间，可以在必要时顺利调动。

师卦的核心哲学是：虽然兵力代表着危险和战争，但其使用应当是圣人不得已而为之的最后手段。在正确的情况下，军事力量可以顺应形势，将危险转化为吉祥。因此，在军事策略上，此卦象征着出征将会顺利，无灾祸。

师卦紧随讼卦（争讼卦），在《序卦》中解释为“讼必有众起，故受之以师”。这意味着争端激化至众多人群的参与，形成了类似军队的集体力量。

根据《象辞》，此卦地中有水，呈现出军队聚众的形象。君子观此卦象，取法于容纳江河的大地，收容和畜养大众。这暗示着在困难重重的大环境下，需要包容别人，充满共情和善意。

师卦的各种解释中，都强调了困难重重、变化无穷的特点，以及对于团结力量和领导者正道的重视。在求名、婚恋和决策方面，也都强调了务实、慎